<a href="https://colab.research.google.com/github/Peritec-Corp/IoTSolution/blob/main/Node_RED_%E3%82%A2%E3%82%AB%E3%82%A6%E3%83%B3%E3%83%88%E8%BF%BD%E5%8A%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# サーバーへのアクセス

PCを同じネットワークに接続し、コマンドラインから以下でアクセスできる。
```
ssh peritec@iotschool
```

リモートからもアクセスできるようにしているがここでは省略\
以下参照\
(個人ファイル\takanami\Setup KVM Server.ipynb パスワード.txt)

---
# vim について

本作業ではvimでファイルを更新するのでそれの最小限の使い方を記録する。

vimは insertモードと visualモードがある。\
insertモードの場合は、下側に```-- INSERT --```と表示されている。\
入力は insertモードの時のみ。

visualモードでの操作
- ```i``` : insertモードに切り替え
- ```:wq```：変更を適用して終了
- ```:q!```：変更を破棄して終了
- ```v``` ->範囲指定-> ```y```：指定した範囲をコピー
- ```p```：ペースト
- ```h```,```j```,```k```,```l```：カーソル移動

insertモードでの操作
- ```esc```：visualモードに切り替え

---
# 設定ファイル更新

```
cd nginx-nr
cp -r s01 [新規ユーザー名]
vim docker-compose.yml #修正内容は下参照
cd nginx
vim default.conf #修正内容は下参照
cd ../[新規ユーザー名]
vim settings.js #修正内容は下参照
```
※ユーザー名は小文字にする

docker-compose.ymlファイルに他のユーザー設定と同じように新規ユーザー情報を追加する。
```
[新規ユーザー名]:
  volumes:
    - type: volume
      source: data_[新規ユーザー名]
      target: /data
  build:
      context : ./[新規ユーザー名]/
      dockerfile: Dockerfile
  container_name: [新規ユーザー名]
  ports:
    - [新規ポート]:[新規ポート]
  extra_hosts:
    - "peritec.net:192.168.254.245"
```
一番下のvolumesにも追加
```
data_[新規ユーザー名]
```

default.confファイルに他のユーザー設定と同じように新規ユーザー情報を追加する。
```
  location /[新規ユーザー名]/ {
    proxy_http_version 1.1;
    proxy_set_header Host              $host;
    proxy_set_header Upgrade $http_upgrade;
    proxy_set_header Connection "upgrade";

    proxy_pass        http://[新規ユーザー名]:[新規ポート]/;
  }
```

settings.jsファイルを更新する
```
username: "[新規ユーザー名]"
password: "[パスワード文字列]"
```
```
uiPort: process.env.PORT || [新規ポート],
```
パスワードの生成は、以下コマンド後にパスワードを入力するとパスワード文字列を生成できる。
```
node-red admin hash-pw
```



設定更新後、Dockerを再起動する。
```
docker compose down --rmi all --volumes --remove-orphans
docker compose up -d --build
```
上記コマンドは省略コマンドも作っている
```
docker-down
docker-up
```
